In [ ]:
from google.colab import drive
from google.colab.patches import cv2_imshow
from torchvision import transforms
drive.mount(('/content/drive/'))

Mounted at /content/drive/


In [ ]:
import os
import torch
import numpy as np
import torchvision
import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import cv2
import matplotlib.pyplot as plt
import math

In [ ]:
from glob import glob
import pandas

In [ ]:
device = 'cuda:0'
num_joints = 21

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F

device = 'cuda:0'


# 2d pose estimator - pretrained
class CPM2DPose(nn.Module):
    def __init__(self):
        super(CPM2DPose, self).__init__()
        
        self.scoremap_list = []
        self.layers_per_block = [2, 2, 4, 2]
        self.out_chan_list = [64, 128, 256, 512]
        self.pool_list = [True, True, True, False]

        self.relu = F.leaky_relu
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_1
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_3 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_4 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_5 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv4_7 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1, bias=True)  # conv0_2
        self.conv5_1 = nn.Conv2d(128, 512, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.conv5_2 = nn.Conv2d(512, 21, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.conv6_1 = nn.Conv2d(149, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv6_2 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv6_3 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv6_4 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv6_5 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv6_6 = nn.Conv2d(128, 128, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.conv6_7 = nn.Conv2d(128, 21, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.conv7_1 = nn.Conv2d(149, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv7_2 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv7_3 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv7_4 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv7_5 = nn.Conv2d(128, 128, kernel_size=7, stride=1, padding=3, bias=True)  # conv0_2
        self.conv7_6 = nn.Conv2d(128, 128, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.conv7_7 = nn.Conv2d(128, 21, kernel_size=1, stride=1, padding=0, bias=True)  # conv0_2
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.conv1_1(x))
        x = self.relu(self.conv1_2(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2_1(x))
        x = self.relu(self.conv2_2(x))
        x = self.maxpool(x)
        x = self.relu(self.conv3_1(x))
        x = self.relu(self.conv3_2(x))
        x = self.relu(self.conv3_3(x))
        x = self.relu(self.conv3_4(x))
        x = self.maxpool(x)
        x = self.relu(self.conv4_1(x))
        x = self.relu(self.conv4_2(x))
        x = self.relu(self.conv4_3(x))
        x = self.relu(self.conv4_4(x))
        x = self.relu(self.conv4_5(x))
        x = self.relu(self.conv4_6(x))
        encoding = self.relu(self.conv4_7(x))
        x = self.relu(self.conv5_1(encoding))
        scoremap = self.conv5_2(x)

        x = torch.cat([scoremap, encoding], 1)
        x = self.relu(self.conv6_1(x))
        x = self.relu(self.conv6_2(x))
        x = self.relu(self.conv6_3(x))
        x = self.relu(self.conv6_4(x))
        x = self.relu(self.conv6_5(x))
        x = self.relu(self.conv6_6(x))
        scoremap = self.conv6_7(x)
        x = torch.cat([scoremap, encoding], 1)
        x = self.relu(self.conv7_1(x))
        x = self.relu(self.conv7_2(x))
        x = self.relu(self.conv7_3(x))
        x = self.relu(self.conv7_4(x))
        x = self.relu(self.conv7_5(x))
        x = self.relu(self.conv7_6(x))
        x = self.conv7_7(x)
        return x


In [ ]:
class Classifier(nn.Module):
    def __init__(self, batch_size):
        super(Classifier, self).__init__()
        self.batch_size = batch_size

        self.fc_layer = nn.Sequential(
            nn.Linear(21*32*32,256),
            nn.ReLU(),
            nn.Linear(256,3)
        )
      
    def forward(self, x):
        x = x.view(self.batch_size, -1)
        x = self.fc_layer(x)
        return x
        

In [ ]:
train_img_len = 100
test_img_len = 10

class HandDataset(Dataset):
    def __init__(self, method=None):
        self.x_data = []
        self.y_data = []
        self.z_data = []
        self.root = '/content/drive/My Drive/hand_posture_data'
        self.weight_root = self.root

        if method == 'train':
            self.root = self.root + '/train/' # self.root = drive/My Drive/hand_posture_data/train/
            rock_path = self.root + 'rock/'
            scissors_path = self.root + 'scissors/'
            paper_path = self.root + 'paper/'

            self.rock_img_path = sorted(glob(rock_path + 'rgb/*.jpg'))
            self.scissors_img_path = sorted(glob(scissors_path + 'rgb/*.jpg'))
            self.paper_img_path = sorted(glob(paper_path + 'rgb/*.jpg'))

            self.img_path = self.rock_img_path + self.scissors_img_path + self.paper_img_path

        elif method == 'test':
            self.root = self.root + '/test/' # self.root = drive/My Drive/hand_posture_data/test/
            rock_path = self.root + 'rock/'
            scissors_path = self.root + 'scissors/'
            paper_path = self.root + 'paper/'

            self.rock_img_path = sorted(glob(rock_path + 'rgb/*.jpg'))
            self.scissors_img_path = sorted(glob(scissors_path + 'rgb/*.jpg'))
            self.paper_img_path = sorted(glob(paper_path + 'rgb/*.jpg'))

            self.img_path = self.rock_img_path + self.scissors_img_path + self.paper_img_path

        for i in tqdm.tqdm(range(len(self.img_path))):
            img = cv2.imread(self.img_path[i], cv2.IMREAD_COLOR)
            #print(self.img_path[i])  
            b, g, r = cv2.split(img)
            img = cv2.merge([r, g, b])
            self.x_data.append(img)

            num = self.img_path[i].split('.')[0].split('/')[-1]
            
            if method == 'train':
              if i in range (train_img_len):
                img_pkl = self.root + 'rock/meta/' + str(num) + '.pkl'
              elif i in range (1*train_img_len, 2*train_img_len):
                img_pkl = self.root + 'scissors/meta/' + str(num) + '.pkl'
              elif i in range(2*train_img_len, 3*train_img_len):
                img_pkl = self.root + 'paper/meta/' + str(num) + '.pkl'

            elif method == 'test':
              if i in range(test_img_len):
                img_pkl = self.root + 'rock/meta/' + str(num) + '.pkl'
              elif i in range(test_img_len, 2*test_img_len):
                img_pkl = self.root + 'scissors/meta/' + str(num) + '.pkl'
              elif i in range(2*test_img_len, 3*test_img_len):
                img_pkl = self.root + 'paper/meta/' + str(num) + '.pkl'

            pkl = pandas.read_pickle(img_pkl)
            coords_2d = pkl['coords_2d']
            # coords_2d의 shape = 21*2
            self.y_data.append(coords_2d)


        length = 0
        if method == 'train':
          length = train_img_len
        elif method == 'test':
          length = test_img_len

        for i in range(3):  # rock = 0, scissors = 1, paper = 2
          for j in range(length):
            self.z_data.append(i)

    def __len__(self):
        return len(self.img_path)

    def __getitem__(self, idx):
        transform1 = torchvision.transforms.ToTensor()
        new_x_data = transform1(self.x_data[idx])

        return new_x_data, self.y_data[idx], self.z_data[idx]
    

In [ ]:
class Trainer(object):
    def __init__(self, posenet_epochs, classifier_epochs, batch_size, posenet_lr, classifier_lr):
        self.posenet_epochs = posenet_epochs
        self.classifier_epochs = classifier_epochs
        self.batch_size = batch_size
        self.posenet_learning_rate = posenet_lr
        self.classifier_learning_rate = classifier_lr
        dataset = HandDataset(method='train')
        self.weight_root = dataset.weight_root # weight_root = drive/My Drive/hand_posture_data
        self._build_model()
        self.root = dataset.root # self.root = drive/My Drive/hand_posture_data/train/
        self.dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True, num_workers=2)

        print("Training...")

    def _build_model(self):
        # 2d pose estimator
        poseNet = CPM2DPose()
        self.poseNet = poseNet.to(device)
        self.poseNet.train() # train 모드 명시
        self.poseNet.load_state_dict(torch.load(self.weight_root+'/pretrained_weight.pth'))
        classifier = Classifier(self.batch_size)
        self.classifier = classifier.to(device)
        self.classifier.train()

        print('Finish build model.')

    def skeleton2heatmap(self, _heatmap, keypoint_targets):
        heatmap_gt = torch.zeros_like(_heatmap, device=_heatmap.device) 
        # _heatmap은 ground_truth heatmap만들때 dimension 같게 해주려고 참고하는 용도로만 사용

        keypoint_targets = (((keypoint_targets)) // 8)
        for i in range(keypoint_targets.shape[0]):
            for j in range(21):
                x = int(keypoint_targets[i, j, 0])
                y = int(keypoint_targets[i, j, 1])
                heatmap_gt[i, j, x, y] = 1

        heatmap_gt = heatmap_gt.detach().cpu().numpy()
        for i in range(keypoint_targets.shape[0]):
            for j in range(21):
                heatmap_gt[i, j, :, :] = cv2.GaussianBlur(heatmap_gt[i, j, :, :], ksize=(3, 3), sigmaX=2, sigmaY=2) * 9 / 1.1772
        heatmap_gt = torch.FloatTensor(heatmap_gt).to(device)
        return heatmap_gt # ground truth heatmap


    def heatmap2skeleton(self, heatmapsPoseNet):
        skeletons = np.zeros((heatmapsPoseNet.shape[0], heatmapsPoseNet.shape[1], 2))
        for m in range(heatmapsPoseNet.shape[0]):
            for i in range(heatmapsPoseNet.shape[1]):
                u, v = np.unravel_index(np.argmax(heatmapsPoseNet[m][i]), (32, 32))
                skeletons[m, i, 0] = u * 8
                skeletons[m, i, 1] = v * 8
        return skeletons


    def train(self):
        cpm2dpose_losses = []
        classifier_losses = []
        loss_func = torch.nn.MSELoss()
        classifier_loss_func = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.SGD(self.poseNet.parameters(), self.posenet_learning_rate)
        classifier_optimizer = torch.optim.SGD(self.classifier.parameters(), self.classifier_learning_rate)
        #classifier_optimizer = torch.optim.Adam(self.classifier.parameters(), lr=self.learning_rate, betas=(0.9, 0.999), eps=1e-08)

        for param in self.poseNet.parameters():
          param.required_grad = True
        
        for epoch in tqdm.tqdm(range(self.posenet_epochs + 1)):
            posenet_loss = 0
            for batch_idx, samples in enumerate(self.dataloader):
                optimizer.zero_grad()
                x_train, y_train, z_train = samples #z_train은 semantic class정보(rock=0/scissors=1/paper=2)

                heatmapsPoseNet = self.poseNet(x_train.cuda())
                gt_heatmap = self.skeleton2heatmap(heatmapsPoseNet, y_train)

                loss = loss_func(heatmapsPoseNet, gt_heatmap)
                posenet_loss += loss.item()
                loss.backward(retain_graph=True)  # backpropagation
                optimizer.step() # update parameters

            print('posenet training loss: ', posenet_loss)
            cpm2dpose_losses.append(posenet_loss)

            if epoch == self.posenet_epochs:
               torch.save(self.poseNet.state_dict(), self.weight_root+'/pretrained_weight.pth')

        for epoch in tqdm.tqdm(range(self.classifier_epochs + 1)):
            classifier_loss = 0
            for batch_idx, samples in enumerate(self.dataloader):
                classifier_optimizer.zero_grad()
                x_train, y_train, z_train = samples #z_train은 semantic class정보(rock=0/scissors=1/paper=2)

                heatmapsPoseNet = self.poseNet(x_train.cuda())

                gt_heatmap = self.skeleton2heatmap(heatmapsPoseNet, y_train.cuda()) # heatmapsPoseNet은 shape 참고하기 위해서 만든것일뿐
                
                pred = self.classifier(gt_heatmap)
                
                loss2 = classifier_loss_func(pred, z_train.cuda())
                loss2.backward()
                classifier_loss += loss2.item()
                classifier_optimizer.step()
            print('classifier training loss: ', classifier_loss)
            classifier_losses.append(classifier_loss)

            if epoch == self.classifier_epochs:
               torch.save(self.classifier.state_dict(), self.weight_root+'/classifier_weight.pth')

        
        plt.subplot(2,1,1)
        plt.plot(cpm2dpose_losses, color = 'r')
        plt.subplot(2,1,2)
        plt.plot(classifier_losses, color = 'b')
        print('Finish training.')


In [ ]:
class Tester(object):
    def __init__(self, batch_size):

        self.batch_size = batch_size
        dataset = HandDataset(method='test')
        self.CPM_weight_root = dataset.weight_root + '/pretrained_weight.pth'
        self._build_model()
        self.root = dataset.root # self.root = drive/My Drive/hand_posture_data/test/
        self.dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False, num_workers=2)

        self.datalen = dataset.__len__()
        self.weight_path = dataset.weight_root
        self.weight_path = self.weight_path+'/classifier_weight.pth' # weight_PATH = drive/My Drive/hand_posture_data/classifier_weight.pth
        self.classifier.load_state_dict(torch.load(self.weight_path))

        print("Testing...")

    def _build_model(self):
        # 2d pose estimator
        poseNet = CPM2DPose()
        self.poseNet = poseNet.to(device)
        self.poseNet.load_state_dict(torch.load(self.CPM_weight_root))
        classifier = Classifier(self.batch_size)
        self.classifier = classifier.to(device)

    def skeleton2heatmap(self, _heatmap, keypoint_targets):
        heatmap_gt = torch.zeros_like(_heatmap, device=_heatmap.device) 

        keypoint_targets = (((keypoint_targets)) // 8)
        for i in range(keypoint_targets.shape[0]):
            for j in range(21):
                x = int(keypoint_targets[i, j, 0])
                y = int(keypoint_targets[i, j, 1])
                heatmap_gt[i, j, x, y] = 1

        heatmap_gt = heatmap_gt.detach().cpu().numpy()
        for i in range(keypoint_targets.shape[0]):
            for j in range(21):
                heatmap_gt[i, j, :, :] = cv2.GaussianBlur(heatmap_gt[i, j, :, :], ksize=(3, 3), sigmaX=2, sigmaY=2) * 9 / 1.1772
        heatmap_gt = torch.FloatTensor(heatmap_gt).to(device)
        return heatmap_gt 

    def test(self):
        correct = 0
        correct_2 = 0
        total = 0
        for batch_idx, samples in enumerate(self.dataloader): 
            x_test, y_test, z_test = samples 
            heatmapsPoseNet = self.poseNet(x_test.cuda())

            pred = self.classifier(heatmapsPoseNet)

            for k in range(self.batch_size):

              total +=1
              if np.argmax(pred[k].detach().cpu().numpy()) == z_test[k]:
                correct+=1
            
            ################################################
            
            gt_heatmap = self.skeleton2heatmap(heatmapsPoseNet, y_test)
            pred_2 = self.classifier(gt_heatmap)


            for q in range(self.batch_size):
              print('my prediction: ', np.argmax(pred_2[q].detach().cpu().numpy()))
              if np.argmax(pred_2[q].detach().cpu().numpy()) == z_test[q]:
                correct_2+=1
            

        print('Classifier Accuracy: ', correct_2/total)    
        print('Overall Accuracy: ', correct/total)
        self.classifier_accuracy = correct_2/total
        self.overall_accuracy = correct/total



In [ ]:
def main():
    
    posenet_epochs = 40
    classifier_epochs = 20
    batchSize = 30
    posenet_learningRate = 1e-2
    classifier_learningRate = 1e-1

    #for epochs in epoch_candidate:
    #  for learningRate in learningRate_candidate:
    trainer = Trainer(posenet_epochs, classifier_epochs, batchSize, posenet_learningRate, classifier_learningRate)
    trainer.train()

    tester = Tester(batchSize)
    tester.test()



if __name__ == '__main__':
    main()

  0%|          | 0/41 [00:00<?, ?it/s]

Finish build model.
Training...


  2%|▏         | 1/41 [00:07<05:14,  7.87s/it]

posenet training loss:  0.05850501684471965


  5%|▍         | 2/41 [00:15<05:09,  7.94s/it]

...

posenet training loss:  0.05805013608187437


 98%|█████████▊| 40/41 [05:41<00:08,  8.60s/it]

posenet training loss:  0.05803913902491331
posenet training loss:  0.058027124498039484


  0%|          | 0/21 [00:00<?, ?it/s]


RuntimeError: ignored